# **Spec**

In [1]:
!nvidia-smi

Thu Sep  9 05:11:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b BM # 자기 브랜치 이름으로 변경
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 291, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (244/244), done.
remote: Total 291 (delta 158), reused 177 (delta 45), pack-reused 0
Receiving objects: 100% (291/291), 5.44 MiB | 1.77 MiB/s, done.
Resolving deltas: 100% (158/158), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 12.3 MB/s 
     |████████████████████████████████| 282 kB 36.3 MB/s 
     |████████████████████████████████| 2.8 MB 50.2 MB/s 
     |████████████████████████████████| 919 kB 34.3 MB/s 
     |████████████████████████████████| 1.2 MB 38.6 MB/s 
     |████████████████████████████████| 302 kB 46.7 MB/s 
     |████████████████████████████████| 119 kB 56.1 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 243 kB 44.9 MB/s 
     |████████████████████████████████| 636 kB 45.5 MB/s 
     |██████████████████████████

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


# 시드 통일하기

In [4]:
import torch
from transformers import set_seed


set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# 모델 불러오기

In [5]:
# 만약 ForSequenceClassification이 붙은 모델을 사용할 경우
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [6]:
from transformers import DistilBertForSequenceClassification
from tokenization_kobert import KoBertTokenizer

model = DistilBertForSequenceClassification.from_pretrained("monologg/distilkobert", **model_config)
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", model_max_length=512)

# model.save_pretrained("./model")
tokenizer.tokenize("안녕하세요. 오늘 날씨는 비가 계속 내리내요.")

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.w

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


['▁안', '녕', '하세요', '.', '▁오늘', '▁날씨', '는', '▁비가', '▁계속', '▁내리', '내', '요', '.']

# 하이퍼 파라미터 설정

In [ ]:
num_classes = 3

# 원하는 대로 고쳐서 사용
batch_size = 32
lr = 0.0001
epochs = 150

In [ ]:
# 옵티마이저와 스케줄러
# 원하는 대로 고쳐서 사용

# from torch.optim import AdamW
# from transformers import AdamW, get_linear_schedule_with_warmup


# LENGTH_OF_TRAIN_DATA = 40242
# num_training_steps = ((LENGTH_OF_TRAIN_DATA - 1) // batch_size + 1) * epochs
# optimizer = AdamW(model.parameters(), lr=lr)
# scheduler = get_linear_schedule_with_warmup(optimizer, int(num_training_steps * 0.1), num_training_steps)

# 만약 이 스케줄러를 사용할 경우,
# optimizer.step() 바로 다음에
# scheduler.step()을 호출해야 함

# f1 score 계산하기

In [ ]:
# 1. torchmetrics 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음

# from torchmetrics import F1


# f1_score = F1(num_classes=num_classes)

# # 검증 단계에서 사용
# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)  # 대충 아웃풋
#     pred = torch.argmax(output, dim=1)
#     batch_f1 = f1_score(pred, labels)
#     print(batch_f1)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 2. datasets 사용
# requirements에 넣어놨으니 따로 설치할 필요 없음
# 3번 사이킷런 사용법 쓰십시오 이건 좋지 않음

# from datasets import load_metric


# f1_score = load_metric("f1")

# for inputs, labels in val_loader:
#     # 대충 코드
#     output = model(inputs)
#     pred = torch.argmax(output, dim=1)
#     f1_score.add_batch(predictions=pred, references=labels)

# f1 = f1_score.compute()
# print(f1)

In [ ]:
# 3. sklearn 사용
# from sklearn.metrics import accuracy_score, f1_score


# def compute_metrics(p):
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "f1": f1}

# **훈련**

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

In [9]:
from data_loader import get_data_loaders


train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)
train_data = train_data.remove_columns("token_type_ids")
val_data = val_data.remove_columns("token_type_ids")
test_data = test_data.remove_columns("token_type_ids")

Using custom data configuration default-2cde95b901fe4f1f
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-2cde95b901fe4f1f/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


  0%|          | 0/45154 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [10]:
model_name = "DistilKoBERT_data_v2"
save_path = f"{path_to_drive}/{model_name}"

In [11]:
from transformers import TrainingArguments, EarlyStoppingCallback

from trainer import ImbalancedSamplerTrainer
 

# optimizer = AdamW(model.parameters(), lr =1e-4, eps = 1e-8)
# num_train_epochs = 150
# scheduler = get_polynomial_decay_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_epochs)

args = TrainingArguments(
    output_dir=save_path,
    learning_rate=1e-04,
    logging_dir=save_path + "/log", 
    lr_scheduler_type="linear", # (:obj:`str` or :class:`~transformers.SchedulerType`, `optional`, defaults to :obj:`"linear"`):The scheduler type to use. See the documentation of :class:`~transformers.SchedulerType` for all possible values.
    save_total_limit = 1,       # Only last 1 models are saved(best model). Older ones are deleted. ref= https://stackoverflow.com/questions/62525680/save-only-best-weights-with-huggingface-transformers
    load_best_model_at_end=True,  
    num_train_epochs=150,       # total number of training epochs
    seed=42,
    fp16=True,
    warmup_ratio=0.1,
    weight_decay=0.01,

    # checkpoint
    overwrite_output_dir=True,
    save_strategy='steps',
    save_steps=1000,

    # evaluation
    evaluation_strategy='steps',
    eval_steps=1000,
    metric_for_best_model="f1",

    per_device_train_batch_size=32,     # batch size per device during training
    per_device_eval_batch_size=32,      # batch size for evaluation
    greater_is_better=True,
)

trainer = ImbalancedSamplerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    # tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15)],
)

Using amp fp16 backend


In [12]:
trainer.train()

***** Running training *****
  Num examples = 45154
  Num Epochs = 150
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 211800


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.963000,0.723464,0.777504,0.464323
2000,0.595300,0.507774,0.797630,0.571056
3000,0.385900,0.456903,0.826029,0.624867
4000,0.272200,0.337373,0.878131,0.723452
5000,0.202000,0.343397,0.881932,0.737385
6000,0.165200,0.360327,0.883274,0.799820
7000,0.135400,0.370672,0.895796,0.817522
8000,0.112400,0.367512,0.903399,0.841870
9000,0.100200,0.350085,0.923971,0.851796
10000,0.086600,0.266352,0.942531,0.859152


***** Running Evaluation *****
  Num examples = 4472
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2/checkpoint-1000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1314: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
***** Running Evaluation *****
  Num examples = 4472
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2/checkpoint-2000
Configuration saved in /

TrainOutput(global_step=72000, training_loss=0.066308930794398, metrics={'train_runtime': 9025.6524, 'train_samples_per_second': 750.428, 'train_steps_per_second': 23.466, 'total_flos': 1.5460811497728e+17, 'train_loss': 0.066308930794398, 'epoch': 50.99})

In [13]:
# trainer.train(resume_from_checkpoint=True) # 끊기기전 모델 사용하여 계속 학습 진행

ValueError: ignored

In [25]:
trainer.save_model(output_dir=save_path)

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2
Configuration saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2/pytorch_model.bin


# **모델 테스트**

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [5]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tokenization_kobert import KoBertTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/DistilKoBERT_data_v2/checkpoint-57000")  # /checkpoint-57000
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", model_max_length=512) 

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [6]:
# 테스트 데이터

test_df = pd.read_csv("all_data_v2.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(50).tolist()

# **스피드 테스트**

In [7]:
model.to("cpu")
model.eval()
print()

In [8]:
%%timeit

with torch.no_grad():
    tokens = tokenizer(speed_test_data, padding="max_length", truncation=True, return_tensors="pt")
    del tokens["token_type_ids"]
    _ = model(**tokens)

1 loop, best of 5: 23.9 s per loop


# **F1 Score**

In [9]:
from data_loader import get_data_loaders


_, _, test_dataset = get_data_loaders(tokenizer, return_loader=False, use_imbalanced=False, device="cuda") # DistilKoBERT Dataset

# _, _, test_loader = get_data_loaders(tokenizer, use_imbalanced=False, device="cuda")

Using custom data configuration default-9a5c2b0c925f1169


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9a5c2b0c925f1169/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

In [10]:
from torch.utils.data import DataLoader

test_dataset = test_dataset.remove_columns("token_type_ids")
test_loader = DataLoader(test_dataset)

In [11]:
from torchmetrics import F1
from tqdm.auto import tqdm


f1_score = F1(num_classes=3, average="macro").cuda()
# f1
model.cuda()
model.eval()

pred_array = []
labels_data = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        labels = batch.pop("labels")
        output = model(**batch)
        logits = output.logits
        pred = torch.argmax(logits, dim=1)
        f1_score(pred, labels)
        pred_list = pred.tolist()
        pred_array.append(pred_list[0])
        labels_list = labels.tolist()
        labels_data.append(labels_list[0])

score = f1_score.compute()
print(score.item())

  0%|          | 0/5000 [00:00<?, ?it/s]

0.9144892692565918
